## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [10]:



mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [11]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [12]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")


In [13]:
from mlflow.entities import ViewType
active_experiments = client.search_experiments(view_type=ViewType.ACTIVE_ONLY)

for exp in active_experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}, Artifact Location: {exp.artifact_location}")


Name: nyc-taxi-experiment, ID: 1, Artifact Location: /Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1
Name: Default, ID: 0, Artifact Location: mlflow-artifacts:/0


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/04/04 20:27:19 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/04/04 20:27:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/2/80c361e7228849229e8c45ffefe253c4/artifacts'
🏃 View run flawless-auk-799 at: http://127.0.0.1:5000/#/experiments/2/runs/80c361e7228849229e8c45ffefe253c4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


### Interacting with the model registry

In [22]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/04/04 20:32:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1743813147736, current_stage='None', description='', last_updated_timestamp=1743813147736, name='iris-classifier', run_id='a3939468b8844bf1bc75b23304858bb8', run_link='', source='/Users/zihaozhou/VS_code/mlops-zoomcamp/02-experiment-tracking/mlruns/1/a3939468b8844bf1bc75b23304858bb8/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>